# KLUE BERT base
---
목표
 - 모델과 데이터를 정상적으로 불러오고, 작동하는 것을 확인
 - Preprocessing을 개선하고, fine-tuning을 통해 모델의 성능을 개선
 - 모델 학습에 Bucketing을 성공적으로 적용하고, 그 결과를 비교분석 

In [1]:
import os
import numpy as np

import tensorflow
import transformers
from transformers import Trainer, TrainingArguments

import datasets
from datasets import load_dataset, load_metric

from transformers import AutoTokenizer, AutoModelForSequenceClassification

## 1. NSMC 데이터 분석 및 Huggingface dataset 구성

In [2]:
nsmc_dataset = load_dataset('nsmc')
print(nsmc_dataset)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/6.33M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset nsmc downloaded and prepared to /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [3]:
train = nsmc_dataset['train']
cols = train.column_names
print(cols)
for i in range(5):
    print(train[i])

['id', 'document', 'label']
{'label': 0, 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'id': '9976970'}
{'label': 1, 'document': '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'id': '3819312'}
{'label': 0, 'document': '너무재밓었다그래서보는것을추천한다', 'id': '10265843'}
{'label': 0, 'document': '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'id': '9045019'}
{'label': 1, 'document': '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 'id': '6483659'}


## 2. klue/bert-base model 및 tokenizer 불러오기

In [4]:
# model & tokenizer load
huggingface_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
huggingface_model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

## 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기

In [5]:
# tokenizer를 사용하여 변환
def transform(data):
    return huggingface_tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False,
        )

for i in range(5):
    print(transform(nsmc_dataset['train'][i]))

{'input_ids': [2, 1376, 831, 2604, 18, 18, 4229, 9801, 2075, 2203, 2182, 4243, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [6]:
# valiadation data 생성
nsmc_data = nsmc_dataset['train']. train_test_split(train_size=0.8)

train_data = nsmc_data['train'].map(transform, batched=True)
val_data = nsmc_data['test'].map(transform, batched=True)
test_data = nsmc_dataset['test'].map(transform, batched=True)

print(train_data[0])

  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

{'label': 0, 'document': '2편이라서 2점 준다..1편에 비해서 너무 떨어진다... 재미도 없고 감동도 없고 그냥 이도저도 아닌 술에 물탄듯', 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [7]:
# training arguments 설정
output_dir = 'transformers'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

In [8]:
metric = load_metric('f1')

def compute_metrics(eval_pred):    
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    return metric.compute(predictions=preds, references=labels)

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

In [9]:
trainer = Trainer(
    model=huggingface_model,     # 학습시킬 model
    args=training_arguments,     # TrainingArguments을 통해 설정한 arguments
    train_dataset=train_data,    # training dataset
    eval_dataset=val_data,       # evaluation dataset
    compute_metrics=compute_metrics,
)

In [10]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Sep 18 02:39:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 120000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 45000


Epoch,Training Loss,Validation Loss,F1
1,0.288100,0.368641,0.895365


Saving model checkpoint to transformers/checkpoint-500
Configuration saved in transformers/checkpoint-500/config.json
Model weights saved in transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to transformers/checkpoint-1000
Configuration saved in transformers/checkpoint-1000/config.json
Model weights saved in transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to transformers/checkpoint-1500
Configuration saved in transformers/checkpoint-1500/config.json
Model weights saved in transformers/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to transformers/checkpoint-2000
Configuration saved in transformers/checkpoint-2000/config.json
Model weights saved in transformers/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to transformers/checkpoint-2500
Configuration saved in transformers/checkpoint-2500/config.json
Model weights saved in transformers/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to transformers/checkpoint-30

Model weights saved in transformers/checkpoint-21000/pytorch_model.bin
Saving model checkpoint to transformers/checkpoint-21500
Configuration saved in transformers/checkpoint-21500/config.json
Model weights saved in transformers/checkpoint-21500/pytorch_model.bin
Saving model checkpoint to transformers/checkpoint-22000
Configuration saved in transformers/checkpoint-22000/config.json
Model weights saved in transformers/checkpoint-22000/pytorch_model.bin
Saving model checkpoint to transformers/checkpoint-22500
Configuration saved in transformers/checkpoint-22500/config.json
Model weights saved in transformers/checkpoint-22500/pytorch_model.bin
Saving model checkpoint to transformers/checkpoint-23000
Configuration saved in transformers/checkpoint-23000/config.json
Model weights saved in transformers/checkpoint-23000/pytorch_model.bin
Saving model checkpoint to transformers/checkpoint-23500
Configuration saved in transformers/checkpoint-23500/config.json
Model weights saved in transformers

In [ ]:
trainer.evaluate(test_data)

In [ ]:
del huggingface_model

## 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기
 - 모델의 정확도를 90% 이상

In [ ]:
# tokenizer를 사용하여 변환
def transform2(data):
    return huggingface_tokenizer(
        data['document'],
        truncation = True,
        batched = True,        
        padding = 'max_length',
        return_token_type_ids = False,
        )

train_data = nsmc_data['train'].map(transform2)
val_data = nsmc_data['test'].map(transform2)
test_data = nsmc_dataset['test'].map(transform2)

In [ ]:
training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 5e-05,                         #learning_rate default 값
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 1,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
    gradient_accumulation_steps=64, # 역전파 단계 수 적용
)

### Fine-tuning Parameters 

https://huggingface.co/docs/transformers/v4.21.2/en/main_classes/trainer#transformers.TrainingArguments

---

 bf16, fp32, fp16 비교
 - bf16
     - fp32보다 메모리 사용량이 작지만 정밀도가 더 낮음 모델 
     - 학습시 fp32 대비 메모리 사용량을 약 50% 줄일 수 있으며, 정확도 손실이 크지 않은 경우에 사용

 - fp32
     - 일반적으로 모델 학습에 사용되는 부동 소수점 형식
     - fp16과 비교하여 정밀도가 높으며, 모델의 정확도를 높일 수 있음
     - 연산 속도가 느리고 메모리 사용량이 크기 때문에 대규모 모델 학습에는 제한적

 - fp16
     - 딥 러닝 추론(inference) 분야에서 많이 사용
     - fp32 대비 연산 속도가 빠르고 메모리 사용량이 적음
     - 정밀도가 낮아서 모델의 정확도가 떨어질 수 있으며, 따라서 학습에는 적합하지 않음
 - Mixed Precision
     - fp16, fp32를 혼합하면서 모델학습에 사용하는 방식
     - 학습에 사용되는 메모리 사용량을 최적화하여 학습을 가속화 하면서도 모델의 정확도를 유지할수있음
     -  일반적으로 Mixed precision은 다음과 같은 과정이 존재
         1. 모델의 가중치(weight)는 fp32 형식으로 저장
         2. 입력 데이터는 fp16 형식으로 변환하여 처리
         3. fp16 형식으로 처리하는 도중에 정밀도가 손실될 가능성이 있으므로, 일정 주기마다 가중치를 fp32로 복사하여 정밀도를 보정
         4. 역전파(backpropagation) 과정에서도 fp16을 사용하여 연산 속도를 높임
         5. 학습이 끝나면, 모델의 가중치를 다시 fp32로 변환하여 저장함
     -  fp32를 사용할때보다 많은 메모리 사용을 줄일수가있어서, 일반적으로 대규모 모델 학습에 사용됨
    
fp32에서 속도가 더 느려서 실행하지 못함

group_by_length
 - 적용되는 패딩을 최소화하고 보다 효율적으로 사용하기 위함
 - 트레이닝 데이터 세트에서 대략 동일한 길이의 샘플들을 그룹화할지 여부
 - 동적 패딩을 적용하는 경우에만 유용

In [ ]:
trainer = Trainer(
    model=huggingface_model,     # 학습시킬 model
    args=training_arguments,     # TrainingArguments을 통해 설정한 arguments
    train_dataset=train_data,    # training dataset
    eval_dataset=val_data,       # evaluation dataset
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_data)

In [ ]:
del huggingface_model

## 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교
 - bucketing과 dynamic padding이 무엇인지 알아보고, 이들을 적용하여 model을 학습
     - https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/data_collator
     - https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
 - 모델 성능 향상과 훈련 시간 두 가지 측면 비교
     - trade-off 관계가 발생하는지 여부를 확인

### bucketing
---
   - 가변 입력 시퀀스 길이 데이터를 다룰 때 사용 
   - 입력 길이에 대해 일정 구간들을 나누어 비슷한 길이를 가진 데이터끼리 처리할 수 있도록 하는 기법
   - ex) 최소 시퀀스 5, 최대 시퀀스 150인 상황이 있다고 할 때 같은 배치 안에 최소와 최대가 섞인 경우가 생길 수 있는데 이 경우 최대에 맞춰 패딩을 하면 비효율적임 따라서 시퀀스 길이 5~50, 50~100, 100~150 이런식으로 구간을 나누어 해당 데이터들을 같이 처리
    

### dynamic padding
---
   - 개별 배치에 대해서 별도로 패딩을 수행하여 과도하게 긴 입력으로 인한 과도한 패딩(padding) 작업을 하면 하면 학습 속도가 상당히 빨라지지만 TPU에서 학습하는 경우 문제가 발생할 수 있음
   - TPU는 추가적인 패딩(padding)이 필요한 경우에도 전체 데이터셋이 고정된 형태를 선호함
   - 이를 수행하려면, 배치(batch)로 분리하려는 데이터셋의 요소 각각에 대해서 정확한 수의 패딩(padding)을 적용할 수 있는 콜레이트 함수(collate function)를 정의해야 함
   - ransformers 라이브러리는 DataCollatorWithPadding을 통해 이러한 기능을 제공

### trade off
---
   - 트레이드오프란 객체의 어느 한부분의 품질을 높이거나 낮추는면 다른 부분의 품질을 높이거나 낮추는데 영향을 끼치는 상황

In [ ]:
# 트랜스포머에서 제공한 padding 사용
from transformers import DataCollatorWithPadding

collator = DataCollatorWithPadding(tokenizer)

In [ ]:
# tokenizer를 사용하여 변환
def transform3(data):
    return huggingface_tokenizer(
        data['document'],
        truncation = True,
        batched = True, 
        return_token_type_ids = False,
        )

train_data = nsmc_data['train'].map(transform3)
val_data = nsmc_data['test'].map(transform3)
test_data = nsmc_dataset['test'].map(transform3)

In [ ]:
training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 5e-05,                         #learning_rate default 값
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 1,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
    gradient_accumulation_steps=64, # 역전파 단계 수 적용
    group_by_length = True,
    data_collator = collator,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_data)

In [ ]:
del huggingface_model

결과
---
